In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    cm_name: Path
    target_column: str

In [8]:
from dnaseq.constants import *
from dnaseq.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        target = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            cm_name=config.cm_name,
            target_column=target.name
        )
        
        return model_evaluation_config

In [9]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
import numpy as np
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        return rmse, mae, r2
    
    def evaluate(self):
        test_data = pd.read_csv('artifacts/data_transformation/test.csv')
        test_x = test_data.drop(self.config.target_column, axis=1).values.astype('U')
        test_y = test_data[self.config.target_column]
        
        estimator = joblib.load(self.config.model_path)
        
        test_x = estimator['preprocessor'].transform(test_x.ravel())
        pred = estimator['classifier'].predict(test_x)
        
        (rmse, mae, r2) = self.eval_metrics(test_y, pred)
        
        scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
        
        cm = confusion_matrix(test_y, pred)
        
        save_json(path=Path(self.config.metric_file_name), data=scores)
        
        with open(self.config.cm_name, 'w') as fw:
            fw.write(str(cm))

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()

except Exception as e:
    raise e

[2024-04-03 16:07:15,622: INFO: common: created directory at: artifacts]
[2024-04-03 16:07:15,624: INFO: common: created directory at: artifacts/model_evaluation]


[2024-04-03 16:07:19,353: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
